In [2]:
import urllib.request
import zipfile
import os
from pathlib import Path
url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

In [3]:
def download_and_unzip_spam_data(
    url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download "
            "and extraction."
        )
        return
    with urllib.request.urlopen(url) as response:
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)

    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as {data_file_path}")

download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

sms_spam_collection/SMSSpamCollection.tsv already exists. Skipping download and extraction.


#### Load into a Pandas DF

In [4]:
import pandas as pd
df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
df

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


#### Examine value distribution

In [5]:
print(df["Label"].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


There is a class inbalance. For ease, we will keep a 50:50 class-split

In [6]:
def create_balanced_dataset(df):
    spam_df = df[df["Label"] == "spam"]
    ham_df = df[df["Label"] == "ham"].sample(n=len(spam_df), random_state=123)
    balanced_df = pd.concat([spam_df, ham_df])
    return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

Label
spam    747
ham     747
Name: count, dtype: int64


Map labels into integers {1, 0}

In [7]:
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})
balanced_df

,Label,Text
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
5,1,FreeMsg Hey there darling it's been 3 week's n...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...
11,1,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
4707,0,Wow so healthy. Old airport rd lor. Cant thk o...
3293,0,Dear good morning how you feeling dear
1278,0,Dont put your phone on silent mode ok
4079,0,Gam gone after outstanding innings.


##### Create random split for training, validation, test (70%, 10%, 20%)

In [8]:
def random_split(df, train_frac=0.7, val_frac=0.1, random_state=123):
    # Shuffle the DataFrame.
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    train_end = int(len(df) * train_frac)
    val_end = int(len(df) * (train_frac + val_frac))
    df_train = df[:train_end]
    df_val = df[train_end:val_end]
    df_test = df[val_end:]
    return df_train, df_val, df_test

df_train, df_val, df_test = random_split(balanced_df)
print(f"Train size: {len(df_train)}")
print(f"Validation size: {len(df_val)}")
print(f"Test size: {len(df_test)}")

Train size: 1045
Validation size: 150
Test size: 299


In [9]:
# Save to CSV files.
df_train.to_csv("train.csv", index=None)
df_val.to_csv("validation.csv", index=None)
df_test.to_csv("test.csv", index=None)

### Create Dataset

We now need to create a dataloader to ingest the data into the LLM. 

Note that the text messages may have different sizes, and thus we will pad all messages to have the same length in a given batch.

In [10]:
import torch
from torch.utils.data import Dataset

class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.pad_token_id = tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"})[0]

        # Encode all texts in the dataset.
        self.encoded_texts = [tokenizer.encode(text) for text in self.data["Text"]]
        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            # Truncate texts that are longer than max_length.
            self.encoded_texts = [enc[:max_length] for enc in self.encoded_texts]
        
        # Pad all encoded texts to max_length.
        self.encoded_texts = [
            enc + [self.pad_token_id] * (self.max_length - len(enc)) for enc in self.encoded_texts
        ]

    def __getitem__(self, idx):
        text = torch.tensor(self.encoded_texts[idx], dtype=torch.long)
        label = torch.tensor(self.data.iloc[idx]["Label"], dtype=torch.long)
        return text, label

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        return max(len(enc) for enc in self.encoded_texts)

##### Load Train, Validation and Test sets into the dataloaders

In [11]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
train_dataset = SpamDataset("train.csv", max_length=None, tokenizer=tokenizer)
print( "Train: Number of tokens in the longest sequence:", train_dataset.max_length)

val_dataset = SpamDataset("validation.csv", max_length=train_dataset.max_length, tokenizer=tokenizer)
test_dataset = SpamDataset("test.csv", max_length=train_dataset.max_length, tokenizer=tokenizer)

Train: Number of tokens in the longest sequence: 120


##### TODO: Exercise 6.1 Increasing the context length
Pad the inputs to the maximum number of tokens the model supports and observe
how it affects the predictive performance

### Creating Data Loaders

In [12]:
from torch.utils.data import DataLoader

batch_size = 8
num_workers = 0
torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=True, 
    drop_last=True
)
val_loader = DataLoader(
    dataset=val_dataset, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False, 
    drop_last=False
)
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False, 
    drop_last=False
)

To get an idea of the batch shapes, let's print them down below:

In [13]:
for input_batch, target_batch in train_loader:
    break
print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions:", target_batch.shape)

Input batch dimensions: torch.Size([8, 120])
Label batch dimensions: torch.Size([8])


To get an idea of the dataset size, let's print it down below:

In [14]:
print(len(train_loader), "batches in the training DataLoader")
print(len(val_loader), "batches in the validation DataLoader")
print(len(test_loader), "batches in the test DataLoader")

130 batches in the training DataLoader
19 batches in the validation DataLoader
38 batches in the test DataLoader


#### Load GPT2 Weights

In [15]:
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12, "file_name": "gpt2-small-124M.pth"},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16, "file_name": "gpt2-medium-355M.pth"},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20, "file_name": "gpt2-large-774M.pth"},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25, "file_name": "gpt2-xl-1558M.pth"},
}

In [16]:
BASE_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True,
}

In [17]:
model_name = "gpt2-xl (1558M)"
file_name = model_configs[model_name]["file_name"]
BASE_CONFIG.update(model_configs[model_name])

Load the model

In [18]:
# Load GPT model
%run gpt-model.ipynb
%run data-processing.ipynb

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
 Jiang exclude intensimet
 B LeavingACC Deng
Mean:
  tensor([-0.3596, -0.2606])
Variance :
  tensor([0.2015, 0.2673])
Norm. Mean:
  tensor([    -0.0000,      0.0000], grad_fn=<MeanBackward1>)
Norm. Variance :
  tensor([1.0000, 1.0000], grad_fn=<VarBackward0>)
tensor([[0.2685, 0.7413],
        [0.2738, 0.7564],
        [0.2668, 0.7366],
        [0.2618, 0.7218],
        [0.2712, 0.7495]], grad_fn=<MmBackward0>)
tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V2 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V1 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.

In [19]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    return torch.tensor(encoded).unsqueeze(0)

def token_ids_to_text(tokens, tokenizer):
    formatted = tokens.squeeze(0).tolist()
    return tokenizer.decode(formatted)

In [20]:
url = f"https://huggingface.co/rasbt/gpt2-from-scratch-pytorch/resolve/main/{file_name}"

if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded to {file_name}")

gpt = GPTModel(BASE_CONFIG)
gpt.load_state_dict(torch.load(file_name, weights_only=True))
gpt.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt.to(device)

GPTModel(
  (tok_emb): Embedding(50257, 1600)
  (pos_emb): Embedding(1024, 1600)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (dropout): Dropout(p=0.0, inplace=False)
      (ff): FeedForwardModule(
        (layers): Sequential(
          (0): Linear(in_features=1600, out_features=6400, bias=True)
          (1): GELU()
          (2): Linear(in_features=6400, out_features=1600, bias=True)
        )
      )
      (att): MultiHeadAttention(
        (W_key): Linear(in_features=1600, out_features=1600, bias=True)
        (W_query): Linear(in_features=1600, out_features=1600, bias=True)
        (W_value): Linear(in_features=1600, out_features=1600, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (out_proj): Linear(in_features=1600, out_features=1600, bias=True)
      )
    )
    (1): TransformerBlock(
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (dropout): Dropout(p=0.0, inplace=False)
   

In [23]:
torch.manual_seed(123)

token_ids = generateText(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=BASE_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you closer… You are all here for us… Please take us to God!" she cried.
But they knew to hold the


Let's test whether the model is good at classifying spam already, before doing any fine-tuning

In [26]:
text_2 = (
    "Is the following text 'spam'? Answer with 'yes' or 'no':"
    " 'You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award.'"
)
token_ids = generateText(
    model=model,
    idx=text_to_token_ids(text_2, tokenizer),
    max_new_tokens=23,
    context_size=BASE_CONFIG["context_length"]
)
print(token_ids_to_text(token_ids, tokenizer))

Is the following text 'spam'? Answer with 'yes' or 'no': 'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.' clans Chester seasonal borrowed elevate tou Sorazek Hats Arkham subreddit manipulated Belle famously appreciation declaresildo lich PowerShell expireucked SHnesday


As you can see, the model is not able to classify the text into spam or not spam, it just autocompletes.

### Adding a Classification Layer

Let's first display the structure of the model.
As expected, the last layer of the model has an output size of ~50k, equivalent to the size of the model vocab.

In [27]:
print(gpt)

GPTModel(
  (tok_emb): Embedding(50257, 1600)
  (pos_emb): Embedding(1024, 1600)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (dropout): Dropout(p=0.0, inplace=False)
      (ff): FeedForwardModule(
        (layers): Sequential(
          (0): Linear(in_features=1600, out_features=6400, bias=True)
          (1): GELU()
          (2): Linear(in_features=6400, out_features=1600, bias=True)
        )
      )
      (att): MultiHeadAttention(
        (W_key): Linear(in_features=1600, out_features=1600, bias=True)
        (W_query): Linear(in_features=1600, out_features=1600, bias=True)
        (W_value): Linear(in_features=1600, out_features=1600, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (out_proj): Linear(in_features=1600, out_features=1600, bias=True)
      )
    )
    (1): TransformerBlock(
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (dropout): Dropout(p=0.0, inplace=False)
   

##### Freeze all weights

In [28]:
for param in model.parameters():
    param.requires_grad = False

##### Replace classification layer

In [29]:
torch.manual_seed(123)

num_classes = 2
# This new layer has requires_grad=True by default.
gpt.out_head = torch.nn.Linear(BASE_CONFIG["emb_dim"], num_classes)

##### Unfreeze last transformer block and final layer norm.

In [30]:
for param in gpt.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in gpt.final_norm.parameters():
    param.requires_grad = True

##### Test the new model output shapes

Let's first create the input dimensions.

In [31]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions: torch.Size([1, 4])


The last output dimension is now `2`

In [32]:
with torch.no_grad():
    outputs = gpt(inputs)
    print("Outputs:\n", outputs)
    print("Outputs dimensions:", outputs.shape)

Outputs:
 tensor([[[ 0.1749,  0.6254],
         [-0.7583,  0.0307],
         [ 0.1332,  0.7561],
         [-0.0606,  0.4415]]])
Outputs dimensions: torch.Size([1, 4, 2])


The first dimension `1` is the batch size, `4` is the token dimension, `2` is the output dimension.

We use the output of the latest token as it is the prediction of the model when it uses all of the tokens as input.

In [33]:
print("Last output token:", outputs[:, -1, :])

Last output token: tensor([[-0.0606,  0.4415]])


In order to obtain the class label, we take the argmax. 

There is no need to apply the softmax as we don't need to know the probability scores.

In [34]:
probas = torch.softmax(outputs[:, -1, :], dim=-1)
label = torch.argmax(probas)

print("Class label:", label.item())

Class label: 1


##### TODO: Exercise 6.3 Fine-tuning the first vs. last token
Try fine-tuning the first output token. Notice the changes in predictive performance
compared to fine-tuning the last output token.

##### Create function to measure loader's accuracy

In [41]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None): 
    correct_predictions, num_examples = 0, 0

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i >= num_batches: break

        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)

        with torch.no_grad():
            logits = model(input_batch)[ :, -1, :]
        predicted_labels = torch.argmax(logits, dim=-1)
        
        num_examples += predicted_labels.shape[0]
        correct_predictions += (predicted_labels == target_batch).sum().item()

    return correct_predictions / num_examples

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "mps"

train_accuracy = calc_accuracy_loader(train_loader, gpt, device, 1)
val_accuracy =  calc_accuracy_loader(val_loader, gpt, device, 1)
test_accuracy =  calc_accuracy_loader(test_loader, gpt, device, 1)

print (f"Training Accuracy : {train_accuracy*100:.2f}%" )
print (f"Val. Accuracy : {val_accuracy*100:.2f}%" )
print (f"Test Accuracy : {test_accuracy*100:.2f}%" )

Training Accuracy : 37.50%
Val. Accuracy : 50.00%
Test Accuracy : 37.50%


We need to implement a loss, to be able to train the classifier.
Accuracy is not differentiable, so we use `cross_entropy`

##### Calculate Loss functions

In [48]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)

    logits = model(input_batch)[:,-1,:]
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss

In [51]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i >= num_batches: break

        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)

        with torch.no_grad():
            total_loss += calc_loss_batch(input_batch, target_batch, model, device)

    return total_loss / num_batches # Average loss

In [52]:
with torch.no_grad():
    train_loss = calc_loss_loader(
        train_loader, model, device, num_batches=5
    )
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)
    test_loss = calc_loss_loader(test_loader, model, device, num_batches=5)

print(f"Training loss: {train_loss:.3f}")
print(f"Validation loss: {val_loss:.3f}")
print(f"Test loss: {test_loss:.3f}")

Training loss: 11.444
Validation loss: 11.320
Test loss: 11.348
